In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## House Prices - Advanced Regression Techniques

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',None)

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train['train_test'] = 1
test['train_test'] = 0
test['SalePrice'] = np.NaN
all_data = pd.concat([train,test])
all_data.head()

In [ ]:
all_data.columns

In [ ]:
all_data.shape

In [ ]:
plt.rcParams['figure.figsize'] = (10,8)

In [ ]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data['LotFrontage'] = all_data['LotFrontage'].fillna(all_data['LotFrontage'].mean())

In [ ]:
all_data.drop(['Alley'],axis=1,inplace=True)

In [ ]:
all_data['Utilities'] = all_data['Utilities'].fillna(all_data['Utilities'].mode()[0])

In [ ]:
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

In [ ]:
all_data['MasVnrType'] = all_data['MasVnrType'].fillna(all_data['MasVnrType'].mode()[0])
all_data['MasVnrArea'] = all_data['MasVnrArea'].fillna(all_data['MasVnrArea'].mean()) 

In [ ]:
all_data.drop(['Id','PoolQC','Fence','MiscFeature'],axis=1,inplace=True)

In [ ]:
df_cat = all_data.select_dtypes(include='O')
df_cat.columns

In [ ]:
for i in df_cat.columns:
       all_data[i] = all_data[i].fillna(all_data[i].mode()[0])

In [ ]:
df_num = all_data.select_dtypes(exclude='O')
df_num.columns

In [ ]:
for i in df_num.columns:
       all_data[i] = all_data[i].fillna(all_data[i].mean())

In [ ]:
all_data.isnull().sum().sum()

In [ ]:
sns.heatmap(all_data.isnull(),yticklabels=False,cbar=False,cmap='coolwarm')

In [ ]:
df_cat.shape

In [ ]:
all_data.columns

In [ ]:
all_dummies = pd.get_dummies(all_data[['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition', 'train_test']])

In [ ]:
all_dummies.shape

In [ ]:
plt.rcParams['figure.figsize'] = (30,30)

In [ ]:
sns.heatmap(all_data.corr(),annot=True)

In [ ]:
all_dummies.head()

In [ ]:
X_train = all_dummies[all_dummies.train_test == 1].drop(['train_test'],axis=1)
X_test = all_dummies[all_dummies.train_test == 0].drop(['train_test'],axis=1)

In [ ]:
y_train = all_data[all_data.train_test == 1].SalePrice
y_train.shape

In [ ]:
from xgboost import XGBRegressor

In [ ]:
clf = XGBRegressor()
clf.fit(X_train,y_train)

In [ ]:
clf.score(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
# final_data = {'Id':test.Id,'SalePrice': y_pred}
# submission = pd.DataFrame(data=final_data)
# submission.to_csv('submission1',index=False)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
clf = XGBRegressor()

In [ ]:
params = {'booster' : ['gbtree','gblinear'],
'base_score' : [0.25,0.5,0.75,1],
'n_estimators' : [100,500,900,1100,1500],
'max_depth' : [2,3,5,10,15],
'learning_rate' : [0.05,0.1,0.15,0.20],
'min_child_weight' : [1,2,3,4]}

In [ ]:
random_cv = RandomizedSearchCV(estimator=clf,param_distributions=params,cv=5,
                               n_iter=50,scoring='neg_mean_absolute_error',
                               n_jobs=-1,verbose=5,return_train_score=True,
                              random_state=42)

In [ ]:
random_cv.fit(X_train,y_train)

In [ ]:
random_cv.best_params_

In [ ]:
random_cv.best_score_

In [ ]:
best_xg = random_cv.best_estimator_

In [ ]:
y_pred = best_xg.predict(X_test)

In [ ]:
best_xg.score(X_train,y_train)

In [ ]:
final_data = {'Id':test.Id,'SalePrice': y_pred}
submission = pd.DataFrame(data=final_data)
submission.to_csv('submission2',index=False)